Install the necessary libraries

In [1]:
#pip install gradio
#pip install openai
#pip install langchain

Import the libraries

In [2]:
import os
from langchain.prompts import PromptTemplate
from openai import OpenAI
import gradio as gr

Create the Prompt Template

In [3]:
prompt_template_pdg_str ="""
Generate a multi paragraph rich text product description based on the information provided in the product name, product type and features separated by commas.
The description should appeal to the target audience. Keep the tone as described in the context section.
Example Format:
PRODUCT NAME: product name here
FEATURES: keywords separated by commas here
PRODUCT DESCRIPTION: product description here

Generate a product description that is creative and SEO compliant. Start!
**Context:
PRODUCT NAME: {product_name}
PRODUCT TYPE: {product_type}
KEY FEATURES: {key_features}
TONE: {tone}
TARGET AUDIENCE: {target_audience}
PRODUCT DESCRIPTION:"""

Defines the parameters of the model

In [5]:
llm_api_key = "lm-studio"
llm_api_base = "http://localhost:1234/v1"
llm_model = "lmstudio-community/gemma-2-9b-it-GGUF"

Create description generation function

In [6]:
def product_desc_generator(product_name, product_type, key_features, target_audience, tone):

    prompt_template_pdg = PromptTemplate(
        input_variables=["product_name", "product_type", "key_features", "target_audience", "tone"], template=prompt_template_pdg_str
    )
    formatted_prompt_pdg = prompt_template_pdg.format(
        product_name=product_name, 
        product_type=product_type, 
        key_features=key_features, 
        target_audience=target_audience, 
        tone=tone)

    llm = OpenAI(
        api_key=llm_api_key,
        base_url=llm_api_base,
    )
    
    chat_response = llm.chat.completions.create(
        model= llm_model,
        messages=[
            {"role": "system", "content": "Act as a copyright expert in product description generation."},
            {"role": "user", "content": formatted_prompt_pdg},
        ], temperature=0.7,
    
    )
    return chat_response.choices[0].message.content


Defining the user interface (chat) in Gradio

In [7]:

with gr.Blocks() as pdgUI:
    gr.HTML("""<h1>AI Product Description Generator</h1>""")
    gr.Markdown("Generate Product Description for your products instantly!<br>")
    with gr.Tab("Request!"):
        product_name = gr.Textbox(
            label="Product brand and model",
            placeholder="...",
        )
        product_type = gr.Textbox(
            label="Product type",
            placeholder="What type of product is it?",
        )
        key_features = gr.Textbox(
            label="Key features",
            placeholder="Detailed list of product features and specifications",
        )        
        target_audience = gr.Textbox(
            label="Target Audience",
            placeholder="Who should be interested in this product?",
        ) 
        tone = gr.Dropdown(
            choices =["formal", "casual", "enthusiastic"],
            label="Tone",
        )      

        click_button = gr.Button(value="Generate!")
        product_description = gr.Textbox(label="Product Description")
        click_button.click(
            product_desc_generator, [product_name, product_type, key_features, target_audience, tone], product_description
        )

pdgUI.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
